<a href="https://colab.research.google.com/github/WhiteTiger-y/Lizmotors-Assignment/blob/main/lizmotors_assignment_sts_llm_with_interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers==4.37.2
!pip install -q bitsandbytes==0.41.3 accelerate==0.25.0
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q gradio
!pip install -q gTTS
!pip install -q huggingface_hub
!pip install -q bark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 21.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.4 MB/s eta 0

In [ ]:
import torch
from transformers import BitsAndBytesConfig, pipeline, AutoTokenizer, AutoModel, AutoProcessor
import whisper
import gradio as gr
import numpy as np
import librosa
import soundfile as sf
from bark import SAMPLE_RATE, generate_audio, preload_models
import warnings

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
# CUDA Check
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using torch {torch.__version__} ({DEVICE})")

Using torch 2.4.0+cu121 (cuda)


In [ ]:
# Load Whisper model for Speech-to-Text
whisper_model = whisper.load_model("medium", device=DEVICE)
print(f"Whisper model loaded with {sum(np.prod(p.shape) for p in whisper_model.parameters()):,} parameters.")


100%|██████████████████████████████████████| 1.42G/1.42G [00:15<00:00, 101MiB/s]


Whisper model loaded with 762,321,920 parameters.


In [ ]:
# Load Text Generation model Google Flan T5 XL
model_id = "google/flan-t5-xl"
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

pipe = pipeline(
    "text2text-generation",
    model=model_id,
    model_kwargs={"quantization_config": quant_config}
)
print(f"Loaded Text Generation model: {model_id}")

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Loaded Text Generation model: google/flan-t5-xl


In [ ]:
# Load Bark model and processor
processor = AutoProcessor.from_pretrained("suno/bark")
bark_model = AutoModel.from_pretrained("suno/bark")

tokenizer_config.json:   0%|          | 0.00/353 [00:00<?, ?B/s]

speaker_embeddings_path.json:   0%|          | 0.00/61.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/8.81k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.49G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/4.91k [00:00<?, ?B/s]

In [ ]:
SPEAKERS = {
    "english-male-1": "v2/en_speaker_1",
    "english-male-2": "v2/en_speaker_2",
    "english-female": "v2/en_speaker_9"
}

In [ ]:
# Convert audio format to 16kHz mono using librosa
def convert_audio_to_whisper_format(audio_path):
    audio, sr = librosa.load(audio_path, sr=16000, mono=True)
    processed_audio_path = "processed_audio.wav"
    sf.write(processed_audio_path, audio, sr)
    return processed_audio_path


In [ ]:
# Transcribe function using Whisper
def transcribe(audio):
    if audio is None or audio == '':
        return ''  # Return empty string if no audio input

    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(whisper_model.device)
    result = whisper.decode(whisper_model, mel)
    return result.text

In [ ]:
def text_to_speech_bark(text, speaker_id, file_path="output.wav"):
    # Use Bark's official generate_audio function with speaker history
    audio_array = generate_audio(text, history_prompt=speaker_id)
    sf.write(file_path, audio_array, 24000)
    return file_path

In [ ]:
def process_audio(audio_path, speaker):
    # Step 1: Transcribe audio
    speech_to_text_output = transcribe(audio_path)

    if not speech_to_text_output.strip():
        return "No speech detected.", "No response generated.", None

    # Step 2: Add instruction to LLM input
    llm_instruction = "You are an AI assistant. Answer the questions asked accurately and concisely. Where ever required please explain in detail"
    llm_input = f"{llm_instruction}\nUser: {speech_to_text_output}"

    # Step 3: Generate LLM response
    llm_response = pipe(llm_input)[0]['generated_text']

    # Step 4: Convert LLM response to speech using Bark
    processed_audio_path = text_to_speech_bark(llm_response, SPEAKERS[speaker])

    return speech_to_text_output, llm_response, processed_audio_path


In [ ]:
def clear_inputs():
    return None, None, None, None

In [ ]:
with gr.Blocks() as demo:
    audio_input = gr.Audio(type="filepath", label="Record your voice")
    speaker_dropdown = gr.Dropdown(choices=list(SPEAKERS.keys()), label="Select Speaker", value="english-male-1")
    transcript_output = gr.Textbox(label="Speech to Text")
    llm_output = gr.Textbox(label="LLM Response")
    audio_output = gr.Audio(label="Response as Audio")

    process_btn = gr.Button("Process Audio")
    clear_btn = gr.Button("Clear")

    process_btn.click(process_audio, inputs=[audio_input, speaker_dropdown], outputs=[transcript_output, llm_output, audio_output])
    clear_btn.click(clear_inputs, outputs=[audio_input, transcript_output, llm_output, audio_output])


In [ ]:
demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://3ace6da1c8ac2520e3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


100%|██████████| 11/11 [00:08<00:00,  1.36it/s]
